In [ ]:
from pydub import AudioSegment,silence
from pydub.playback import play
import speech_recognition as sr
r = sr.Recognizer()

minSilenceLen = 1200
lengthAtLeast = 12 # in Seconds...
audiobookMp3Path = "/Users/joker/Jupyter Workspace/pydub/The Sovereign Individual/02. Preface.mp3"

def split(filepath):
    sound = AudioSegment.from_mp3(filepath)
    dBFS = sound.dBFS
    chunks = silence.split_on_silence(sound, 
        min_silence_len = minSilenceLen,
        silence_thresh = dBFS-16,
        keep_silence = True
    )
    return chunks

chunks = split(audiobookMp3Path)
# print(chunks)
print(len(chunks))

In [ ]:
# code from
# https://stackoverflow.com/questions/45526996/split-audio-files-using-silence-detection

target_length = lengthAtLeast * 1000 #setting minimum length of each chunk to 25 seconds
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length,
        # we can start a new one
        output_chunks.append(chunk)

# print(output_chunks)
print(len(output_chunks))

In [ ]:
for i, chunk in enumerate(output_chunks):
    j = f'{i+1}'
    chunk.export(f"{i+1}.wav".format("wav"), bitrate="48k", format="wav")
    # chunk.export(f"{i}.mp3".format("mp3"), bitrate="48k", format="mp3")

print("Done!")

In [ ]:
wholetext=[]
for i in range(1, len(output_chunks)+1):
    with sr.AudioFile(f'{i}.wav') as source:
        # listen for the data (load audio to memory)
        audio_data = r.record(source)
        # recognize (convert from speech to text)
        text = r.recognize_google(audio_data)
        wholetext.append(text)
        print(f'\n{i}\n{text}\n')

print("\nDone!")

In [ ]:
html =[]

htmlhead="""
<!DOCTYPE html>
<html>
<style>
	body {
		padding: 50px 100px;
		font-family: 'Helvetica Neue UltraLight', Georgia, 'Times New Roman', Times, serif;
		font-size: 16px;
		line-height: 2.2em;
		background-color: #F1F3F4;
	}
	blockquote {
		border-left: 2px solid #ddd;
		padding-left: 25px;
	}
    em {
        font-color: #719f79;
    }
</style>
<body>
"""

htmlfooter="""
</body>
</html>
"""

html.append(htmlhead)

for i, text in enumerate(wholetext):
    text = f"""
<p><em>{i+1}</em>
<blockquote>
<p>
{text}
</p>
<p>
<audio controls loop>
    <source src="{i+1}.wav"
    type="audio/wav">
</audio>
</p>
</blockquote>
</p>
    """
    html.append(text)
    
html.append(htmlfooter)

htmlfile = open("02. Preface - audio-with-text.html", "w")
for line in html:
    htmlfile.write(line)
htmlfile.close()

print("Done!")
